In [2]:
import pandas as pd
import numpy as np
import os
import requests
import bs4
from itertools import repeat, product
from joblib import Parallel, delayed
import joblib
from tqdm.auto import tqdm
import traceback as tb

# Onshore-County ids

In [3]:
## we got this from inspect elements from html code of http://webapps.rrc.state.tx.us/PDQ/generalReportAction.do, onshore county dropdown
with open("onshore_county_ids.txt",'r') as f:
    content = f.readlines()

In [4]:
content

['<option value="001">ANDERSON</option>\n',
 '<option value="003">ANDREWS</option>\n',
 '<option value="005">ANGELINA</option>\n',
 '<option value="007">ARANSAS</option>\n',
 '<option value="009">ARCHER</option>\n',
 '<option value="011">ARMSTRONG</option>\n',
 '<option value="013">ATASCOSA</option>\n',
 '<option value="015">AUSTIN</option>\n',
 '<option value="017">BAILEY</option>\n',
 '<option value="019">BANDERA</option>\n',
 '<option value="021">BASTROP</option>\n',
 '<option value="023">BAYLOR</option>\n',
 '<option value="025">BEE</option>\n',
 '<option value="027">BELL</option>\n',
 '<option value="029">BEXAR</option>\n',
 '<option value="031">BLANCO</option>\n',
 '<option value="033">BORDEN</option>\n',
 '<option value="035">BOSQUE</option>\n',
 '<option value="037">BOWIE</option>\n',
 '<option value="039">BRAZORIA</option>\n',
 '<option value="041">BRAZOS</option>\n',
 '<option value="043">BREWSTER</option>\n',
 '<option value="045">BRISCOE</option>\n',
 '<option value="047">B

In [5]:
pd.Series(content).str.split('>').str[1].str.split('<').str[0]

0      ANDERSON
1       ANDREWS
2      ANGELINA
3       ARANSAS
4        ARCHER
         ...   
249        WOOD
250      YOAKUM
251       YOUNG
252      ZAPATA
253      ZAVALA
Length: 254, dtype: object

In [6]:
pd.Series(content).str.split('=').str[1].str.split('>').str[0].str.strip('"')

0      001
1      003
2      005
3      007
4      009
      ... 
249    499
250    501
251    503
252    505
253    507
Length: 254, dtype: object

In [7]:
county_map = dict(zip(pd.Series(content).str.split('=').str[1].str.split('>').str[0].str.strip('"'),pd.Series(content).str.split('>').str[1].str.split('<').str[0]))
county_map

{'001': 'ANDERSON',
 '003': 'ANDREWS',
 '005': 'ANGELINA',
 '007': 'ARANSAS',
 '009': 'ARCHER',
 '011': 'ARMSTRONG',
 '013': 'ATASCOSA',
 '015': 'AUSTIN',
 '017': 'BAILEY',
 '019': 'BANDERA',
 '021': 'BASTROP',
 '023': 'BAYLOR',
 '025': 'BEE',
 '027': 'BELL',
 '029': 'BEXAR',
 '031': 'BLANCO',
 '033': 'BORDEN',
 '035': 'BOSQUE',
 '037': 'BOWIE',
 '039': 'BRAZORIA',
 '041': 'BRAZOS',
 '043': 'BREWSTER',
 '045': 'BRISCOE',
 '047': 'BROOKS',
 '049': 'BROWN',
 '051': 'BURLESON',
 '053': 'BURNET',
 '055': 'CALDWELL',
 '057': 'CALHOUN',
 '059': 'CALLAHAN',
 '061': 'CAMERON',
 '063': 'CAMP',
 '065': 'CARSON',
 '067': 'CASS',
 '069': 'CASTRO',
 '071': 'CHAMBERS',
 '073': 'CHEROKEE',
 '075': 'CHILDRESS',
 '077': 'CLAY',
 '079': 'COCHRAN',
 '081': 'COKE',
 '083': 'COLEMAN',
 '085': 'COLLIN',
 '087': 'COLLINGSWORTH',
 '089': 'COLORADO',
 '091': 'COMAL',
 '093': 'COMANCHE',
 '095': 'CONCHO',
 '097': 'COOKE',
 '099': 'CORYELL',
 '101': 'COTTLE',
 '103': 'CRANE',
 '105': 'CROCKETT',
 '107': 'CROSBY'

# get first data

In [10]:
# getting fresh JSESSIONID
s = requests.Session()
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # 'Cookie': '_ga=GA1.3.1330622583.1681476442; _gid=GA1.3.910762609.1681476442; JSESSIONID=4V2AH2hx4hTPKo_zr376z59Pc6XEN01oWt99YYR8kffRCiQVNTfn!991815829',
    'Referer': 'http://webapps.rrc.state.tx.us/PDQ/home.do',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.39',
}

response = s.get(
    'http://webapps.rrc.state.tx.us/PDQ/generalReportAction.do',
#     cookies=cookies,
    headers=headers,
    verify=False,
)
suggest = s.get('http://webapps.rrc.state.tx.us/PDQ/generalReportAction.do')

JSESSIONID = s.cookies.get("JSESSIONID")

print("JSESSIONID",JSESSIONID)
print("cookies",s.cookies)

JSESSIONID sVmBLR71clTcEzhkPKdPxkZxU-k4CAMZDPj8K2TWNBasaGLaVxl0!-419651290
cookies <RequestsCookieJar[<Cookie JSESSIONID=sVmBLR71clTcEzhkPKdPxkZxU-k4CAMZDPj8K2TWNBasaGLaVxl0!-419651290 for webapps.rrc.state.tx.us/>]>


In [11]:
cookies = {
    '_ga': 'GA1.3.1330622583.1681476442',
    '_gid': 'GA1.3.910762609.1681476442',
    'JSESSIONID': JSESSIONID, ## get this fresh session id
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    # 'Cookie': '_ga=GA1.3.1330622583.1681476442; _gid=GA1.3.910762609.1681476442; JSESSIONID=VCl_znWA5_hmBrn90Y--1XPv5qSwr2fb_Rx2qXDvU60loP3GE91U!991815829',
    'Origin': 'http://webapps.rrc.state.tx.us',
    'Referer': 'http://webapps.rrc.state.tx.us/PDQ/generalReportAction.do',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.39',
}

data = {
    'viewType': 'Operator',
    'startMonth': '01',
    'startYear': '2022',
    'endMonth': '01',
    'endYear': '2022',
    'wellType': 'Both',
    'district': 'None Selected',
    'onShoreCounty': '001', ## 001 for Anderson 003 for Andrews
    'offShoreArea': 'None Selected',
    'fieldNameData': 'None Selected',
    'fieldRegion': 'None Selected',
    'operatorNameData': 'None Selected',
    'operatorValue': 'None Selected',
    'submit': 'Submit',
}

response = requests.post(
    'http://webapps.rrc.state.tx.us/PDQ/mainReportAction.do',
    cookies=cookies,
    headers=headers,
    data=data,
    verify=False,
)

In [12]:
response.status_code

200

In [13]:
response.text

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<html>\r\n<head>\r\n    <link rel="stylesheet" type="text/css" href="styles/ogm_style3.css">\r\n    <!--<script defer type="text/javascript" src="../js/pngfix.js"></script>-->\r\n    <title>Operator Query Results</title>\r\n</head>\r\n<body>\r\n<!-- System Header -->\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!--Main System Header-->\r\n<table width="100%" border="0" cellspacing="0" cellpadding="0">\r\n<script>\r\nfunction validateHeader(){\r\nif(document.forms["HeaderForm"].elements["go"].selectedIndex ==0)\r\n   {\r\n   \treturn false;\r\n   }else{\r\n   \treturn true;\r\n   }\r\n}\r\n</script>\r\n  <form name="HeaderForm" action="/WFC/submitHeaderAction.do">\r\n  <tr>\r\n      <td class="MainHeader"><img src="/PDQ/images/rrc_online_trans_new.gif" height="35" width="334" border="0" alt=""></td>\r\n      <td align="right" nowrap class="MainHeader">\r\n      <input type="hidden" name="ogmid" value="KbqFJRcI98YXtIjkxCAhY4AjA89zhajdb7OO4AEELgM=">\r\n<!--

In [14]:
df = pd.read_html(response.text)

In [15]:
df

[    0   1   2
 0 NaN NaN NaN,
     0                                1
 0 NaN  Oil & Gas Production Data Query,
     0                                              1
 0 NaN                                            NaN
 1 NaN                  Production Data FAQs PDQ Help
 2 NaN                                            NaN
 3 NaN  General Production Query Specific Lease Query,
                  0   1     2   3         4   5
 0  Production Data NaN  FAQs NaN  PDQ Help NaN,
                           0   1                     2   3
 0  General Production Query NaN  Specific Lease Query NaN,
                                                    0
 0  General Production Query Results Related Links...,
             Related Links
 0           O&G Directory
 1  O&G Proration Schedule
 2     Offshore County Map,
              0                                  1
 0  Query Path:  Search Criteria > County ANDERSON
 1  Date Range:                                NaN,
     0   1                   

In [16]:
df[12]

,Operator Name,Operator No.,Oil (BBL),Casinghead (MCF),GW Gas (MCF),Condensate (BBL)
0,84 ENERGY LLC,856467,0,0,402,1
1,ALLISON/HOBBS INC.,14573,0,0,0,0
2,AMPLIFY ENERGY OPERATING LLC,20467,0,0,1821,17
3,ARMOR RESOURCES LLC,31587,0,0,0,0
4,"ATLANTIS OIL COMPANY, INC.",36562,0,0,0,0
5,ATMOS PIPELINE - TEXAS,36599,0,0,0,0
6,"BARBEE, R.L. OPERATING, INC.",49145,0,0,0,0
7,"BASA RESOURCES, INC.",53974,933,0,808,35
8,"BAY-TECH OPERATING CO., INC.",57935,1177,1042,0,0
9,"BBC&D ENTERPRISES, LLC",58903,0,0,0,0


# Full page

In [43]:

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # 'Cookie': '_ga=GA1.3.1330622583.1681476442; _gid=GA1.3.910762609.1681476442; JSESSIONID=VCl_znWA5_hmBrn90Y--1XPv5qSwr2fb_Rx2qXDvU60loP3GE91U!991815829',
    'Referer': 'http://webapps.rrc.state.tx.us/PDQ/mainReportAction.do',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.39',
}

params = {
    'pagesize': '5000',
}

response = requests.get(
    'http://webapps.rrc.state.tx.us/PDQ/changePageViewAction.do',
    params=params,
    cookies=cookies,
    headers=headers,
    verify=False,
)

In [44]:
response.status_code

200

In [45]:
response.text

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<html>\r\n<head>\r\n    <link rel="stylesheet" type="text/css" href="styles/ogm_style3.css">\r\n    <!--<script defer type="text/javascript" src="../js/pngfix.js"></script>-->\r\n    <title>Operator Query Results</title>\r\n</head>\r\n<body>\r\n<!-- System Header -->\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!--Main System Header-->\r\n<table width="100%" border="0" cellspacing="0" cellpadding="0">\r\n<script>\r\nfunction validateHeader(){\r\nif(document.forms["HeaderForm"].elements["go"].selectedIndex ==0)\r\n   {\r\n   \treturn false;\r\n   }else{\r\n   \treturn true;\r\n   }\r\n}\r\n</script>\r\n  <form name="HeaderForm" action="/WFC/submitHeaderAction.do">\r\n  <tr>\r\n      <td class="MainHeader"><img src="/PDQ/images/rrc_online_trans_new.gif" height="35" width="334" border="0" alt=""></td>\r\n      <td align="right" nowrap class="MainHeader">\r\n      <input type="hidden" name="ogmid" value="KbqFJRcI98YXtIjkxCAhY6vjcaArJjUG+IHdaVjrZnU=">\r\n<!--

In [46]:
df = pd.read_html(response.text)

In [47]:
df

[    0   1   2
 0 NaN NaN NaN,
     0                                1
 0 NaN  Oil & Gas Production Data Query,
     0                                              1
 0 NaN                                            NaN
 1 NaN                  Production Data FAQs PDQ Help
 2 NaN                                            NaN
 3 NaN  General Production Query Specific Lease Query,
                  0   1     2   3         4   5
 0  Production Data NaN  FAQs NaN  PDQ Help NaN,
                           0   1                     2   3
 0  General Production Query NaN  Specific Lease Query NaN,
                                                    0
 0  General Production Query Results Related Links...,
             Related Links
 0           O&G Directory
 1  O&G Proration Schedule
 2     Offshore County Map,
              0                                 1
 0  Query Path:  Search Criteria > County ANDREWS
 1  Date Range:                               NaN,
     0   1                      

In [50]:
df[12].iloc[:-1]

,Operator Name,Operator No.,Oil (BBL),Casinghead (MCF),GW Gas (MCF),Condensate (BBL)
0,1836 ENERGY LLC,623407.0,0,0,0,0
1,"1894 PETROLEUM PARTNERS, LLC",194375.0,95,0,0,0
2,ADKINS PRODUCTION,7920.0,0,0,0,0
3,"AGHORN OPERATING, INC.",8716.0,1484,403,0,0
4,ALLEDER INC.,13296.0,1438,0,0,0
...,...,...,...,...,...,...
188,WTG EXPLORATION,945220.0,4585,5214,250,0
189,XOG OPERATING LLC,945921.0,136,89,0,0
190,XTO ENERGY INC.,945936.0,315151,259407,0,0
191,"ZAGAVA, LLC",950456.0,149,7,0,0


# Automate extraction

In [17]:
# getting fresh JSESSIONID
s = requests.Session()
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # 'Cookie': '_ga=GA1.3.1330622583.1681476442; _gid=GA1.3.910762609.1681476442; JSESSIONID=4V2AH2hx4hTPKo_zr376z59Pc6XEN01oWt99YYR8kffRCiQVNTfn!991815829',
    'Referer': 'http://webapps.rrc.state.tx.us/PDQ/home.do',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.39',
}

response = s.get(
    'http://webapps.rrc.state.tx.us/PDQ/generalReportAction.do',
#     cookies=cookies,
    headers=headers,
    verify=False,
)
suggest = s.get('http://webapps.rrc.state.tx.us/PDQ/generalReportAction.do')

JSESSIONID = s.cookies.get("JSESSIONID")

print("JSESSIONID",JSESSIONID)
print("cookies",s.cookies)

JSESSIONID MBWBLW39aOq68HeDYUY5r0PLvwfzVB7IJzK7Zbo4zzA7C9qxcvZM!-1362717959
cookies <RequestsCookieJar[<Cookie JSESSIONID=MBWBLW39aOq68HeDYUY5r0PLvwfzVB7IJzK7Zbo4zzA7C9qxcvZM!-1362717959 for webapps.rrc.state.tx.us/>]>


In [18]:
cookies = {
    '_ga': 'GA1.3.1330622583.1681476442',
    '_gid': 'GA1.3.910762609.1681476442',
    'JSESSIONID': JSESSIONID, ## get this fresh session id
}

In [19]:
headers = [
   {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded',
    # 'Cookie': '_ga=GA1.3.1330622583.1681476442; _gid=GA1.3.910762609.1681476442; JSESSIONID=VCl_znWA5_hmBrn90Y--1XPv5qSwr2fb_Rx2qXDvU60loP3GE91U!991815829',
    'Origin': 'http://webapps.rrc.state.tx.us',
    'Referer': 'http://webapps.rrc.state.tx.us/PDQ/generalReportAction.do',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.39',
},

{
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # 'Cookie': '_ga=GA1.3.1330622583.1681476442; _gid=GA1.3.910762609.1681476442; JSESSIONID=VCl_znWA5_hmBrn90Y--1XPv5qSwr2fb_Rx2qXDvU60loP3GE91U!991815829',
    'Referer': 'http://webapps.rrc.state.tx.us/PDQ/mainReportAction.do',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.39',
}
    
]

urls = ['http://webapps.rrc.state.tx.us/PDQ/mainReportAction.do',
       'http://webapps.rrc.state.tx.us/PDQ/changePageViewAction.do']

data = {
    'viewType': 'Operator',
    'startMonth': '01',
    'startYear': '2023',
    'endMonth': '01',
    'endYear': '2023',
    'wellType': 'Both',
    'district': 'None Selected',
    'onShoreCounty': '003', ## 001 for Anderson 003 for Andrews
    'offShoreArea': 'None Selected',
    'fieldNameData': 'None Selected',
    'fieldRegion': 'None Selected',
    'operatorNameData': 'None Selected',
    'operatorValue': 'None Selected',
    'submit': 'Submit',
}

params = {
    'pagesize': '10000',
}

verify = False

In [20]:
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul','Aug','Sep','Oct','Nov','Dec']
year = ['2022','2023']

In [21]:
month_map = dict(zip(months,month_names))
month_map

{'01': 'Jan',
 '02': 'Feb',
 '03': 'Mar',
 '04': 'Apr',
 '05': 'May',
 '06': 'Jun',
 '07': 'Jul',
 '08': 'Aug',
 '09': 'Sep',
 '10': 'Oct',
 '11': 'Nov',
 '12': 'Dec'}

In [22]:
items = list(product(year,months,list(county_map)))
items

[('2022', '01', '001'),
 ('2022', '01', '003'),
 ('2022', '01', '005'),
 ('2022', '01', '007'),
 ('2022', '01', '009'),
 ('2022', '01', '011'),
 ('2022', '01', '013'),
 ('2022', '01', '015'),
 ('2022', '01', '017'),
 ('2022', '01', '019'),
 ('2022', '01', '021'),
 ('2022', '01', '023'),
 ('2022', '01', '025'),
 ('2022', '01', '027'),
 ('2022', '01', '029'),
 ('2022', '01', '031'),
 ('2022', '01', '033'),
 ('2022', '01', '035'),
 ('2022', '01', '037'),
 ('2022', '01', '039'),
 ('2022', '01', '041'),
 ('2022', '01', '043'),
 ('2022', '01', '045'),
 ('2022', '01', '047'),
 ('2022', '01', '049'),
 ('2022', '01', '051'),
 ('2022', '01', '053'),
 ('2022', '01', '055'),
 ('2022', '01', '057'),
 ('2022', '01', '059'),
 ('2022', '01', '061'),
 ('2022', '01', '063'),
 ('2022', '01', '065'),
 ('2022', '01', '067'),
 ('2022', '01', '069'),
 ('2022', '01', '071'),
 ('2022', '01', '073'),
 ('2022', '01', '075'),
 ('2022', '01', '077'),
 ('2022', '01', '079'),
 ('2022', '01', '081'),
 ('2022', '01', 

In [23]:
items = [item for item in items if not (item[0]=='2023' and int(item[1])>2)]
print(items)

[('2022', '01', '001'), ('2022', '01', '003'), ('2022', '01', '005'), ('2022', '01', '007'), ('2022', '01', '009'), ('2022', '01', '011'), ('2022', '01', '013'), ('2022', '01', '015'), ('2022', '01', '017'), ('2022', '01', '019'), ('2022', '01', '021'), ('2022', '01', '023'), ('2022', '01', '025'), ('2022', '01', '027'), ('2022', '01', '029'), ('2022', '01', '031'), ('2022', '01', '033'), ('2022', '01', '035'), ('2022', '01', '037'), ('2022', '01', '039'), ('2022', '01', '041'), ('2022', '01', '043'), ('2022', '01', '045'), ('2022', '01', '047'), ('2022', '01', '049'), ('2022', '01', '051'), ('2022', '01', '053'), ('2022', '01', '055'), ('2022', '01', '057'), ('2022', '01', '059'), ('2022', '01', '061'), ('2022', '01', '063'), ('2022', '01', '065'), ('2022', '01', '067'), ('2022', '01', '069'), ('2022', '01', '071'), ('2022', '01', '073'), ('2022', '01', '075'), ('2022', '01', '077'), ('2022', '01', '079'), ('2022', '01', '081'), ('2022', '01', '083'), ('2022', '01', '085'), ('2022', '

In [41]:
dfs = []
failed = []
for yr, mo, coun in tqdm(items):
    try:
        data_ = data.copy()
        data_['startMonth'] = mo
        data_['startYear'] = yr
        data_['endMonth'] = mo
        data_['endYear'] = yr
        data_['onShoreCounty'] = coun


        response = requests.post(
            url=urls[0],
            cookies=cookies,
            headers=headers[0],
            data=data_,
            verify=verify,
        )

        response = requests.get(
            url=urls[1],
            params=params,
            cookies=cookies,
            headers=headers[1],
            verify=verify,
        )

        html_text_response = response.text
        df_ = pd.read_html(html_text_response)
        df_ = df_[12].iloc[:-1]
        df_.insert(loc = 0,column = 'year',value = yr)
        df_.insert(loc = 0,column = 'month',value = mo)
        df_.insert(loc = 0,column = 'county',value = coun)
        dfs.append(df_)
    except KeyboardInterrupt:
        break
    except:
        print(f"Failed | year: {yr} | month: {month_map[mo]} | county: {county_map[coun]}")
        failed.append((yr,mo,coun,tb.format_exc()))
    
comb_df = pd.concat(dfs,ignore_index=True)
comb_df = comb_df.iloc[:,:-1]

Failed | year: 2022 | month: Jan | county: HIDALGO
Failed | year: 2022 | month: Jun | county: MCCULLOCH



In [50]:
comb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205380 entries, 0 to 205379
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   county             205380 non-null  object 
 1   month              205380 non-null  object 
 2   year               205380 non-null  object 
 3   Operator Name      205380 non-null  object 
 4   Operator No.       205380 non-null  float64
 5   Oil  (BBL)         205380 non-null  float64
 6   Casinghead  (MCF)  205380 non-null  float64
 7   GW Gas  (MCF)      205380 non-null  float64
 8   Condensate  (BBL)  205380 non-null  float64
dtypes: float64(5), object(4)
memory usage: 14.1+ MB


In [59]:
comb_df.county = comb_df.county.map(county_map)
comb_df.month = comb_df.month.map(month_map)

In [35]:
comb_df['Operator No.'] = comb_df['Operator No.'].astype(int).astype(str).str.zfill(6)

In [36]:
comb_df

,county,month,year,Operator Name,Operator No.,Oil (BBL),Casinghead (MCF),GW Gas (MCF),Condensate (BBL)
0,ANDERSON,Jan,2022,84 ENERGY LLC,856467,0.0,0.0,402.0,1.0
1,ANDERSON,Jan,2022,ALLISON/HOBBS INC.,014573,0.0,0.0,0.0,0.0
2,ANDERSON,Jan,2022,AMPLIFY ENERGY OPERATING LLC,020467,0.0,0.0,1821.0,17.0
3,ANDERSON,Jan,2022,ARMOR RESOURCES LLC,031587,0.0,0.0,0.0,0.0
4,ANDERSON,Jan,2022,"ATLANTIS OIL COMPANY, INC.",036562,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
205375,ZAVALA,Feb,2023,U.S. ENERGY DEVELOPMENT CORP,875462,0.0,0.0,0.0,0.0
205376,ZAVALA,Feb,2023,VERDUN OIL & GAS LLC,884566,0.0,0.0,0.0,0.0
205377,ZAVALA,Feb,2023,"VICEROY PETROLEUM, LP",885041,0.0,0.0,0.0,0.0
205378,ZAVALA,Feb,2023,"WHITE MARLIN OPERATING CO, LLC",918475,0.0,0.0,0.0,0.0


In [60]:
# comb_df.to_csv('county_operator_prod_data_2022-23.csv',index=False)

In [33]:
# comb_df = pd.read_csv('county_operator_prod_data_2022-23.csv')

In [62]:
df_failed = pd.DataFrame(failed,columns=['year','month','county','traceback'])
df_failed

,year,month,county,traceback
0,2022,01,215,"Traceback (most recent call last):\n File ""<i..."
1,2022,06,307,"Traceback (most recent call last):\n File ""<i..."


In [63]:
# df_failed.to_csv('failed_api_calls.csv',index=False)

In [64]:
for item in failed:
    print(item[-1])
    print("---------------------\n")

Traceback (most recent call last):
  File "<ipython-input-41-661769e40d9a>", line 31, in <module>
    df_ = df_[12].iloc[:-1]
IndexError: list index out of range

---------------------

Traceback (most recent call last):
  File "<ipython-input-41-661769e40d9a>", line 31, in <module>
    df_ = df_[12].iloc[:-1]
IndexError: list index out of range

---------------------



# Failed retry
- run only if failed entries

In [74]:
dfs_failed = []
failed_failed = []
for yr, mo, coun, _ in tqdm(failed):
    try:
        data_ = data.copy()
        data_['startMonth'] = mo
        data_['startYear'] = yr
        data_['endMonth'] = mo
        data_['endYear'] = yr
        data_['onShoreCounty'] = coun


        response = requests.post(
            url=urls[0],
            cookies=cookies,
            headers=headers[0],
            data=data_,
            verify=verify,
        )

        response = requests.get(
            url=urls[1],
            params=params,
            cookies=cookies,
            headers=headers[1],
            verify=verify,
        )

        html_text_response = response.text
        df_ = pd.read_html(html_text_response)
        df_ = df_[12].iloc[:-1]
        df_.insert(loc = 0,column = 'year',value = yr)
        df_.insert(loc = 0,column = 'month',value = mo)
        df_.insert(loc = 0,column = 'county',value = coun)
        dfs_failed.append(df_)
    except KeyboardInterrupt:
        break
    except:
        print(f"Failed | year: {yr} | month: {month_map[mo]} | county: {county_map[coun]}")
        failed_failed.append((yr,mo,coun,tb.format_exc()))
    
comb_df_failed = pd.concat(dfs_failed,ignore_index=True)
# comb_df_failed = comb_df_failed.iloc[:,:-1]

In [75]:
comb_df_failed

,county,month,year,Operator Name,Operator No.,Oil (BBL),Casinghead (MCF),GW Gas (MCF),Condensate (BBL)
0,215,01,2022,2 D OIL & GAS LLC,953828.0,0,0,0,0
1,215,01,2022,ANDERSON OIL LTD.,22499.0,0,0,877,2
2,215,01,2022,ATLAS OPERATING LLC,36587.0,0,0,19607,36
3,215,01,2022,AURORA RESOURCES CORPORATION,36931.0,0,0,0,0
4,215,01,2022,"BEACH EXPLORATION, INC.",58915.0,1341,0,0,0
...,...,...,...,...,...,...,...,...,...
80,307,06,2022,"REBCO OIL, INC.",696214.0,46,0,0,0
81,307,06,2022,SIERRA BLANCA OPERATING,780194.0,0,0,0,0
82,307,06,2022,"SMITH, CHARLES M.",790250.0,0,0,0,0
83,307,06,2022,"STRAWN PARTNERS OPERATING, LLC",825811.0,0,0,0,0


In [76]:
comb_df_failed.county = comb_df_failed.county.map(county_map)
comb_df_failed.month = comb_df_failed.month.map(month_map)

In [ ]:
comb_df_failed['Operator No.'] = comb_df_failed['Operator No.'].astype(int).astype(str).str.zfill(6)

In [77]:
comb_df_failed

,county,month,year,Operator Name,Operator No.,Oil (BBL),Casinghead (MCF),GW Gas (MCF),Condensate (BBL)
0,HIDALGO,Jan,2022,2 D OIL & GAS LLC,953828.0,0,0,0,0
1,HIDALGO,Jan,2022,ANDERSON OIL LTD.,22499.0,0,0,877,2
2,HIDALGO,Jan,2022,ATLAS OPERATING LLC,36587.0,0,0,19607,36
3,HIDALGO,Jan,2022,AURORA RESOURCES CORPORATION,36931.0,0,0,0,0
4,HIDALGO,Jan,2022,"BEACH EXPLORATION, INC.",58915.0,1341,0,0,0
...,...,...,...,...,...,...,...,...,...
80,MCCULLOCH,Jun,2022,"REBCO OIL, INC.",696214.0,46,0,0,0
81,MCCULLOCH,Jun,2022,SIERRA BLANCA OPERATING,780194.0,0,0,0,0
82,MCCULLOCH,Jun,2022,"SMITH, CHARLES M.",790250.0,0,0,0,0
83,MCCULLOCH,Jun,2022,"STRAWN PARTNERS OPERATING, LLC",825811.0,0,0,0,0


In [78]:
comb_fin = pd.concat([comb_df,comb_df_failed],ignore_index=True)

In [30]:
comb_fin['Operator No.'] = comb_fin['Operator No.'].astype(int).astype(str).str.zfill(6)

In [31]:
comb_fin

,county,month,year,Operator Name,Operator No.,Oil (BBL),Casinghead (MCF),GW Gas (MCF),Condensate (BBL)
0,ANDERSON,Jan,2022,84 ENERGY LLC,856467,0.0,0.0,402.0,1.0
1,ANDERSON,Jan,2022,ALLISON/HOBBS INC.,014573,0.0,0.0,0.0,0.0
2,ANDERSON,Jan,2022,AMPLIFY ENERGY OPERATING LLC,020467,0.0,0.0,1821.0,17.0
3,ANDERSON,Jan,2022,ARMOR RESOURCES LLC,031587,0.0,0.0,0.0,0.0
4,ANDERSON,Jan,2022,"ATLANTIS OIL COMPANY, INC.",036562,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
205460,MCCULLOCH,Jun,2022,"REBCO OIL, INC.",696214,46.0,0.0,0.0,0.0
205461,MCCULLOCH,Jun,2022,SIERRA BLANCA OPERATING,780194,0.0,0.0,0.0,0.0
205462,MCCULLOCH,Jun,2022,"SMITH, CHARLES M.",790250,0.0,0.0,0.0,0.0
205463,MCCULLOCH,Jun,2022,"STRAWN PARTNERS OPERATING, LLC",825811,0.0,0.0,0.0,0.0


In [32]:
# comb_fin.to_csv('county_operator_prod_data_2022-23_final.csv',index=False)

In [24]:
# comb_fin = pd.read_csv('county_operator_prod_data_2022-23_final.csv')